In [2]:
import json
from dataset import Drive360Loader
import pandas as pd

#Config to load non downsampled un-normalized test data
config = json.load(open('./config - load test data 2020-04-22.json'))
test_loader = Drive360Loader(config, 'test')

df = test_loader.drive360.dataframe.loc[test_loader.drive360.indices]
df.reset_index(inplace=True, drop=True)

results_to_plot = [
    ('resnet34.csv', 'resnet34'),
    ('resnext101_32x4d_ssl.csv', 'resnext101_32x4d_ssl'),
    ('resnext101_32x4d_swsl.csv', 'resnext101_32x4d_swsl'),
    ('simclr.csv', 'simclr')
]

for results, label in results_to_plot:
    pred = pd.read_csv('evaluate_model_all_bottom_frozen/results/' + results)
    df['pred_speed_{}'.format(label)] = pred['canSpeed']
    df['pred_steer_{}'.format(label)] = pred['canSteering']

df.rename(columns={
    'canSpeed': 'true_speed',
    'canSteering': 'true_steer'
}, inplace=True)

Loading data from ./Drive360Images_160_90/drive360challenge_split_test_stratify.csv...
Phase: test # of data: 136080


In [3]:
#Calculate MSE for each model on entire test data and store in it a csv
from sklearn.metrics import mean_squared_error

models_list = ['resnet34','resnext101_32x4d_ssl','resnext101_32x4d_swsl', 'simclr']
mse_steer_list = []
mse_speed_list = []
y_true_steering = df['true_steer'].to_numpy()
y_true_speed = df['true_speed'].to_numpy()

for label in models_list:
    print("Model - ", label)
    #MSE for steering
    pred_steering = df['pred_steer_{}'.format(label)].to_numpy()
    mse_steer = mean_squared_error(y_true_steering, pred_steering)
    mse_steer_list.append(mse_steer)
    
    #MSE for speed
    pred_speed = df['pred_speed_{}'.format(label)].to_numpy()
    mse_speed = mean_squared_error(y_true_speed, pred_speed)
    mse_speed_list.append(mse_speed)
    
print("MSE steering - ", mse_steer_list)
print("MSE speed - ", mse_speed_list)
df = pd.DataFrame(list(zip(models_list,mse_steer_list,mse_speed_list)), 
                  columns=['Model name','MSE Steering','MSE speed'])
df.to_csv("evaluate_model_all_bottom_frozen/models_mse_on_entire_test_dataset.csv")

Model -  resnet34
Model -  resnext101_32x4d_ssl
Model -  resnext101_32x4d_swsl
Model -  simclr
MSE steering -  [602.6839107226288, 571.6572160168778, 595.0946423372643, 581.10731467566]
MSE speed -  [7.912877217399591, 6.942681341754661, 7.128119002236442, 7.519118052253404]
